In [30]:
from opendbc.car import structs
from opendbc.car.values import PLATFORMS

print(f"Found {len(PLATFORMS)} qualifying vehicles:")
# for platform in PLATFORMS:
#   print(f"  {platform}")

Found 221 qualifying vehicles:


In [31]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 3  # Increase this to search more segments

print("Collecting segments from commaCarSegments dataset:")
for platform in TEST_PLATFORMS:
  if platform not in database:
    print(f"No segments available for for {platform}")
    continue
  
  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  # print(f"Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments")
  TEST_SEGMENTS.extend(random.sample(all_segments, NUM_SEGMENTS))

print(f"Collected {len(TEST_SEGMENTS)} segments for analysis")


No segments available for for DODGE_DURANGO
No segments available for for FORD_RANGER_MK2
No segments available for for HOLDEN_ASTRA
No segments available for for CADILLAC_ATS
No segments available for for CHEVROLET_MALIBU
No segments available for for CADILLAC_XT4
No segments available for for CHEVROLET_VOLT_2019
No segments available for for CHEVROLET_TRAVERSE
No segments available for for GMC_YUKON
No segments available for for HONDA_ODYSSEY_CHN
No segments available for for HYUNDAI_KONA_2022
No segments available for for HYUNDAI_NEXO_1ST_GEN
No segments available for for GENESIS_GV70_ELECTRIFIED_1ST_GEN
No segments available for for GENESIS_G80_2ND_GEN_FL
No segments available for for RIVIAN_R1_GEN1
No segments available for for SUBARU_FORESTER_HYBRID
No segments available for for TESLA_MODEL_3
No segments available for for TESLA_MODEL_Y
No segments available for for TOYOTA_RAV4_PRIME
No segments available for for TOYOTA_SIENNA_4TH_GEN
No segments available for for LEXUS_LC_TSS2
No

In [33]:
from openpilot.selfdrive.pandad import can_capnp_to_list
from openpilot.tools.lib.logreader import LogReader

message_to_check = 0x3c0  # Volkswagen CAN ignition message

segment_count = len(TEST_SEGMENTS)
for idx, segment in enumerate(TEST_SEGMENTS):
  lr = LogReader(segment)
  CP = lr.first("carParams")
  if CP is None:
    continue

  can_packets = [msg for msg in lr if msg.which() == "can"]

  # print(f"Evaluating segment {idx} of {segment_count} for {CP.carFingerprint}")

  matched_segments = {}
  for packet in can_packets:
    for msg in packet.can:
      if msg.address == message_to_check:
        matched_segments.update({segment: CP.carFingerprint})

  if segment in matched_segments:
    print(f"Match found in {segment} for {CP.carFingerprint}")

print(f"Analysis finished")


Match found in d753e5427226080e/2023-12-30--10-14-03/9/s for CHRYSLER PACIFICA HYBRID 2018
Match found in 810855dc1450c716/2023-12-26--10-03-20/3/s for CHRYSLER PACIFICA HYBRID 2018
Match found in 7f268b92fc672c4d/2023-11-02--20-54-34/103/s for CHRYSLER PACIFICA HYBRID 2018
Match found in 1a10807c60c127fd/2023-12-09--19-21-30/49/s for CHRYSLER PACIFICA HYBRID 2019
Match found in b70b56b76a6217f2/2023-12-26--12-29-24/4/s for CHRYSLER PACIFICA HYBRID 2019
Match found in 86deeb65f10c3129/2023-12-29--07-38-12/179/s for CHRYSLER PACIFICA HYBRID 2019
Match found in 0a3e89f78b1d0071/2023-11-30--10-33-13/13/s for CHRYSLER PACIFICA 2018
Match found in 8fc6a1b72c8b1357/2023-11-30--17-35-04/10/s for CHRYSLER PACIFICA 2018
Match found in d5de6f3e9371e89e/2023-11-16--17-19-54/48/s for CHRYSLER PACIFICA 2018
Match found in 5fc1717be83fc0da/2024-01-03--13-16-03/27/s for CHRYSLER PACIFICA 2020
Match found in c1feb10d20626c65/2023-12-16--17-46-24/9/s for CHRYSLER PACIFICA 2020
Match found in c1feb10d20

Match found in 54ada937747ef8ca/2023-12-14--08-14-41/8/s for SKODA SUPERB 3RD GEN
Match found in 54ada937747ef8ca/2023-11-16--06-01-05/7/s for SKODA SUPERB 3RD GEN
Analysis finished
